<a href="https://colab.research.google.com/github/dongchanlim/CS450/blob/master/Week_11_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
file = files.upload()

Saving crx.data to crx.data


In [59]:
credit = pd.read_csv("crx.data", header = None, skipinitialspace= True, na_values= ["?"])
credit.columns = ['A' + str(i) for i in range(1,17)]

- Attribute Description    
  - A1:	b, a.
  - A2:	continuous.
  - A3:	continuous.
  - A4:	u, y, l, t.
  - A5:	g, p, gg.
  - A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
  - A7:	v, h, bb, j, n, z, dd, ff, o.
  - A8:	continuous.
  - A9:	t, f.
  - A10:	t, f.
  - A11:	continuous.
  - A12:	t, f.
  - A13:	g, p, s.
  - A14:	continuous.
  - A15:	continuous.
  - A16: +,-         (class attribute)

In [60]:
credit.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [53]:
credit.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [61]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

num_col = credit.select_dtypes(["number"]).columns

credit[num_col] = std.fit_transform(credit[num_col])

In [62]:
credit.shape

(690, 16)

In [63]:
credit.isnull().sum(axis = 0) / credit.shape[0] * 100

A1     1.739130
A2     1.739130
A3     0.000000
A4     0.869565
A5     0.869565
A6     1.304348
A7     1.304348
A8     0.000000
A9     0.000000
A10    0.000000
A11    0.000000
A12    0.000000
A13    0.000000
A14    1.884058
A15    0.000000
A16    0.000000
dtype: float64

In [64]:
from sklearn.impute import SimpleImputer


si = SimpleImputer(strategy = "most_frequent")

missing_col = ['A1','A2','A4','A5','A6','A7','A14']

credit[missing_col] = si.fit_transform(credit[missing_col])

In [65]:
credit.isnull().sum(axis = 0)

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

In [68]:
from sklearn.model_selection import train_test_split

X = credit.drop(columns = 'A16')
y = credit.A16

X = pd.get_dummies(X)


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

1. Try at least 3 different "regular" learning algorithms and note the results.

In [76]:
# KNN classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors = np.int((np.sqrt(credit.size))), weights = "distance", n_jobs = -1)

knn.fit(X_train, y_train)

predicted_y = knn.predict(X_test)

accuracy_score(predicted_y, y_test)

0.8115942028985508

In [77]:
# Naive Bayesian
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train, y_train)

predicted_y = nb.predict(X_test)

accuracy_score(predicted_y, y_test)

0.6714975845410628

In [79]:
# SVM
from sklearn.svm import SVC

svm = SVC(gamma='auto', kernel = 'sigmoid')

svm.fit(X_train, y_train)

predicted_y = svm.predict(X_test)

accuracy_score(predicted_y, y_test)

0.8647342995169082

2. Use a random forest and note the results. (Play around with a few different options)


In [93]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', max_depth = 4, n_jobs = -1, random_state = 0, class_weight = "balanced_subsample")

rfc.fit(X_train, y_train)

# predicted_y = rfc.predict(X_test)
# accuracy_score(predicted_y, y_test)

rfc.score(X_test, y_test)

0.8405797101449275

In [102]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 150, 200], 'criterion': ["gini",'entropy'], 'max_depth': [2,3,4,5], 'class_weight': ['balanced', 'balanced_subsample'], 'n_jobs' : [-1]}

search = GridSearchCV(rfc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 5,
 'n_estimators': 50,
 'n_jobs': -1}

In [104]:
search.best_score_

0.8963273195876289

3. Use a gradient boosting machine (GBM) and note the results. (Play around with a few different options)


In [92]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.5, max_depth= 4, random_state= 0)

gbc.fit(X_train, y_train)

gbc.score(X_test, y_test)

0.855072463768116

In [107]:
param_grid = {'criterion': ['friedman_mse', 'mse', 'mae'], 'loss' :['deviance', 'exponential'], 'max_features':['auto', 'sqrt', 'log2'], 'max_depth': [2,3,4,5]}

search = GridSearchCV(gbc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

{'criterion': 'friedman_mse',
 'loss': 'exponential',
 'max_depth': 5,
 'max_features': 'log2'}

In [108]:
search.best_score_

0.8943298969072165